# Project Development Notebook 
## Project group name: Lamaceratops_Salsa_Largo
#### Project group members: Jaweria Amjad, Albert Berdjour, Gerald Yiran, Timon Kunze
#### Pod TA: Nazimur Rashid Chowdhury
#### Project TA: Wil Laura

In [1]:
# You can start coding here
#from IPython.display import IFrame
#link_id = "wx7tu"

datamatrix 1.0.1 requires openpyxl, which is not installed.

datamatrix 1.0.1 requires psutil, which is not installed.

datamatrix 1.0.1 requires tomlkit, which is not installed.

geopandas 0.13.2 requires fiona>=1.8.19, which is not installed.

In [2]:
# To solve xesmf error
import os
from pathlib import Path

if 'ESMFMKFILE' not in os.environ:
    os.environ['ESMFMKFILE'] = str(Path(os.__file__).parent.parent / 'esmf.mk')

import xesmf

In [3]:
# imports
import time
tic = time.time()
from scipy.ndimage.measurements import label
import sys
import pandas as pd
import intake
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
import pyleoclim as pyleo

from xmip.preprocessing import combined_preprocessing
from xarrayutils.plotting import shaded_line_plot
import geoviews as gv
import holoviews
from geoviews import Dataset as gvDataset
import geoviews.feature as gf
from geoviews import Image as gvImage

from datatree import DataTree
from xmip.postprocessing import _parse_metric
import cartopy as cart
import cartopy.crs as ccrs
import random
import pooch
import os
import tempfile

import geopy
import geopy.distance
import datetime
from datetime import date, timedelta

/tmp/ipykernel_444794/4104914918.py:4: DeprecationWarning: Please use `label` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  from scipy.ndimage.measurements import label


ModuleNotFoundError: No module named 'xmip'

In [ ]:
# helper functions

def pooch_load(filelocation=None,filename=None,processor=None):
    #shared_location='/home/jovyan/shared/Data/Projects/Heatwaves' # this is different for each day
    shared_location='./data/Projects/Heatwaves' 
    user_temp_cache=tempfile.gettempdir()
    
    if os.path.exists(os.path.join(shared_location,filename)):
        file = os.path.join(shared_location,filename)
    else:
        file = pooch.retrieve(filelocation,known_hash=None,fname=os.path.join(user_temp_cache,filename),processor=processor)

    return file


In [ ]:
# @title Figure settings
import ipywidgets as widgets  # interactive display

%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle")
# model_colors = {k:f"C{ki}" for ki, k in enumerate(source_ids)}
%matplotlib inline

In [ ]:
import intake_esm
print(intake_esm.__version__)

import sys
print(sys.executable)

## CMIP6: 

We will utilize a CMIP6 dataset to examine trends in surface temperature, precipitation and soil moisture on heatwaves.

Specifically, in this project we will will analyze data from a model and examine its historical temperature records.

In [ ]:
# loading CMIP data

col = intake.open_esm_datastore(
    "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
)  # open an intake catalog containing the Pangeo CMIP cloud data

# pick our five example models
# There are many more to test out! Try executing `col.df['source_id'].unique()` to get a list of all available models
source_ids = ['MPI-ESM1-2-LR']#['MRI-ESM2-0']
variable_ids = ["tas"]#["tas", "pr"]
# sorted(col.df["source_id"].unique())


### Fetch data

In [ ]:
# from the full `col` object, create a subset using facet search
cat = col.search(
    source_id=source_ids,
    variable_id= variable_ids,
    member_id="r1i1p1f1",
    table_id="3hr",
    grid_label="gn",
    experiment_id=["historical"],  # add scenarios if interested in projection
    require_all_on=[
        "source_id"
    ],  # make sure that we only get models which have all of the above experiments
)

# convert the sub-catalog into a datatree object, by opening each dataset into an xarray.Dataset (without loading the data)
kwargs = dict(
    preprocess=combined_preprocessing,  # apply xMIP fixes to each dataset
    xarray_open_kwargs=dict(
        use_cftime=True
    ),  # ensure all datasets use the same time index
    storage_options={
        "token": "anon"
    },  # anonymous/public authentication to google cloud storage
)

cat.esmcat.aggregation_control.groupby_attrs = ["source_id", "experiment_id"]
dt = cat.to_datatree(**kwargs)
dt

In [ ]:
# select just a single model and experiment
tas_historical = dt[source_ids[0]]["historical"].ds.tas
print("The time range is:")
print(
    tas_historical.time[0].data.astype("M8[h]"),
    "to",
    tas_historical.time[-1].data.astype("M8[h]"),
)

### Q1: Plot the worldwide near surface temperature distribution (map) for four example years using the CMIP6 dataset.

In [ ]:
fig, ([ax_1980, ax_1990], [ax_2000, ax_2010]) = plt.subplots(
    ncols=2, nrows=2, figsize=[12, 6], subplot_kw={"projection": ccrs.Robinson()}
)

    
# plot a timestep for 2023
tas_1980 = tas_historical.sel(time=slice("1980-01-01", "1980-12-31")).mean(dim="time").squeeze()
tas_1980.plot(
    ax=ax_1980,
    transform=ccrs.PlateCarree(),
    cmap="magma",
    robust=True,
)
ax_1980.coastlines()
ax_1980.set_title("1980")

# repeat for 1990
tas_1990 = tas_historical.sel(time=slice("1990-01-01", "1990-12-31")).mean(dim="time").squeeze()
tas_1990.plot(
    ax=ax_1990,
    transform=ccrs.PlateCarree(),
    cmap="magma",
    robust=True,
)
ax_1990.coastlines()
ax_1990.set_title("1990")

# repeat for 2000
tas_2000 = tas_historical.sel(time=slice("2000-01-01", "2000-12-31")).mean(dim="time").squeeze()
tas_2000.plot(
    ax=ax_2000,
    transform=ccrs.PlateCarree(),
    cmap="magma",
    robust=True,
)
ax_2000.coastlines()
ax_2000.set_title("2000")


tas_2010 = tas_historical.sel(time=slice("2010-01-01", "2010-12-31")).mean(dim="time").squeeze()
tas_2010.plot(
    ax=ax_2010,
    transform=ccrs.PlateCarree(),
    cmap="magma",
    robust=True,
)
ax_2010.coastlines()
ax_2010.set_title("2010")

### Q2: Mask out the ocean, and plot the global mean temperature (yearly average) as a function of time for the past 50 years. Optionally you can also add soil moisture and/or precipitation.

In [ ]:
holoviews.extension("bokeh")
tas_yearly_50 = tas_historical.sel(time=slice("1961-01-01", "2010-12-31")).groupby("time.year").mean()
tas_plot = gvDataset(
    tas_yearly_50.isel(year=slice(0, 2))
)  # first 5 time steps
tas_yearly_50
images = tas_plot.to(
    gvImage, ["y", "x"], ["tas"], "year"
)
images.opts(
    cmap="coolwarm",
    colorbar=True,
    width=600,
    height=400,
    projection=ccrs.Robinson(),
    clabel="Temperature",

) * gf.coastline


### Detect heat days for specific date (global) 

In [ ]:
def plot_global_temperature_map(ds):
    fig, ax_present = plt.subplots(
            figsize=[12, 6], subplot_kw={"projection": ccrs.Robinson()}
        )
    ds.plot(ax=ax_present, transform=ccrs.PlateCarree(), cmap="magma", robust=True)
    ax_present.coastlines()
    ax_present.add_feature(cart.feature.OCEAN, zorder=100, edgecolor='k')
    ax_present.set_title("")
    plt.show()

w_day = 10
tas_selected = tas_historical.sel(time=slice("2000-01-01", "2001-12-31")).squeeze()
tas_daily_mean = tas_selected.resample(time='D').mean(dim='time')
tas_rolling_P95 = tas_daily_mean.rolling(time=w_day, center=True).mean().chunk(time=-1).quantile(.95, dim='time')
tas_grtr_P95 = tas_daily_mean>tas_rolling_P95 # Create Boolean mask
tas_heat_cells = tas_daily_mean.where(tas_grtr_P95).sel(time="2000-07-03").squeeze()

plot_global_temperature_map(tas_heat_cells)

### Q7: Heat wave detection in region

In [ ]:
def compute_tasAndP95mask(tas_historical, year_range):
    """
    computes global temperatures and P95 masks
    """
    
    tas_grtr_P95_all = []
    tas_daily_mean_all = []
    for year in year_range:
        year_start = f"{year}-01-01"
        year_end = f"{year}-12-31"

        # Compute daily mean from
        tas_selected = tas_historical.sel(time=slice(year_start, year_end)).squeeze()
        tas_daily_mean_yrly = tas_selected.resample(time='D').mean(dim='time')
        tas_rolling_P95_yrly = tas_daily_mean_yrly.rolling(time=w_day, center=True).mean().chunk(time=-1).quantile(.95, dim='time')
        tas_grtr_P95_yrly = tas_daily_mean_yrly>tas_rolling_P95_yrly
        tas_grtr_P95_all.append(tas_grtr_P95_yrly)
        tas_daily_mean_all.append(tas_daily_mean_yrly)

    return tas_daily_mean_all, tas_grtr_P95_all
# Given a day, this function computes if the region of interest experienced heatwave

def get_distance_km(lat1, lon1, lat2, lon2):
    """Calculate distance in kilometers between two lat-long points.
    """
    # Create two points
    A = geopy.Point(lat1, lon1)
    B = geopy.Point(lat2, lon2)

    # Calculate the distance between the two points
    dist_km = geopy.distance.distance(A, B).km
    
    return dist_km 


def get_weights(ds):
    """For a rectangular grid the cosine of the latitude 
    is proportional to the grid cell area.
    """
    weights = np.cos(np.deg2rad(ds.lat))
    weights.name = "weights"
 
    return weights


def get_max_cell_area_kmsqr(ds):
    """Calculate the difference in square km between two
    neightboring cells that have a weight of 1 (largest).
    """
    # Get geo-distance between random neighboring points on 
    # lat=-0.9326 which is axis of largest cells
    lat1 = -0.9326
    lon1 = 360.0
    lat2 = lat1
    lon2 = 1.875

    #weights.sel(y=lat1, x=lon1, method="nearest") 
    #weights.sel(y=-lat2, x=lon2, method="nearest")

    max_cell_length = get_distance_km(lat1, lon1, lat2, lon2)
    
    # area as sqared length
    max_cell_area = max_cell_length**2
    
    return max_cell_area


def get_area_of_cell_kmsqr(ds, cell_lat, cell_lon):
    """Since weights have values from 0 to 1, we can
    get the cell km^2-area by weighting with max 
    cell area.
    """
    area = get_weights(ds)*get_max_cell_area_kmsqr(ds)
    area.name = "area"
 
    cell_area = area.sel(y=cell_lat, x=cell_lon, method="nearest")

    return cell_area


def heatwave_over_region(tas_region, 
                         P95_mask_region, 
                         E, X, 
                         start_date=(2010, 7, 15), 
                         plotting=True,
                         mask_ocean=False,
                         verbose=True,
                         ret = False
                        ):
    """
    tas_region: daily mean temperature of the region of interest for the year "year_ofinterest"
    P95_mask_region: mask of the region of interest for the year "year_ofinterest"
    E: minimum area (km^2) with tas>P95 for heatwave
    X: minmum number of days over which tas>P95
    """
    
    lat, lon, time = P95_mask_region.indexes.values()
    # year, month, day = start_date
    end_date = start_date + timedelta(days = X-1)
    if (end_date.year>start_date.year):
        print("Can't detect heatwaves for days that span more than a year")
        return 0
    # print((end_date.strftime('%Y-%m-%d')))
    tas_X_days = P95_mask_region.sel(time=slice(start_date.strftime('%Y-%m-%d'),end_date.strftime('%Y-%m-%d')))
    tas_X_days_numpy = np.nan_to_num(tas_X_days.as_numpy())
    ht_array, num_ht = label(np.prod(tas_X_days_numpy, axis=0))
    # print(ht_array)
    for ii in range(num_ht):
        coords = np.argwhere(ht_array==(ii+1))        
        cell_lats = lat[coords[:, 0]]
        cell_lons = lon[coords[:, 1]]
        
        # dst = tas_daily_mean.sel(time="1980").squeeze()  
        dst = tas_daily_mean.sel(time=str(year_ofinterest)).squeeze() 
        # 1980, 2000, start_date.strftime('%Y-%m-%d')
        area = 0
        for cell_lon in cell_lons:
            for cell_lat in cell_lats:
                area += get_area_of_cell_kmsqr(dst,
                                               cell_lat=cell_lat, 
                                               cell_lon=cell_lon)
        
        #print(area)
        if (area<E):
            for c_i in coords[:, 0]:
                for c_j in coords[:, 1]:
                    ht_array[c_i, c_j] = 0

    if verbose:               
        print(f"Number of heat waves recorded on the date {start_date}: {len(np.unique(ht_array))-1}")
    # uncomment if want to see output temperature array
    # print(np.nan_to_num(tas_region.sel(time=start_date.strftime('%Y-%m-%d')).squeeze().where(ht_array).as_numpy()))
    if plotting:
        fig, ax_present = plt.subplots(figsize=[8, 4], subplot_kw={"projection": ccrs.Robinson()})
        tas_region.sel(time=start_date.strftime('%Y-%m-%d')).squeeze().where(ht_array).plot(ax=ax_present, transform=ccrs.PlateCarree(), 
                                                                                            cmap="magma", robust=True)
        ax_present.coastlines()
        if mask_ocean:
            ax_present.add_feature(cart.feature.OCEAN, zorder=100, edgecolor='k')
        ax_present.set_title(f"Heatwaves on {start_date.strftime('%Y-%m-%d')}")
        plt.show()
    if ret:
        return len(np.unique(ht_array))-1

### Plotting heat cells over X days in region of interest

In [ ]:
# - we count total number of cells in a given region for a number of consecutive days (count_cells_all_days)
# - we count the number of heat cells (i.e. cells that show temperature above the 95% quantile (count_heat_cells_all_days)
# - the ratio between heat cells and total number of cells (count_heat_cells_all_days / count_cells_all_days) is compared to a threshold value
# - if the ratio is higher than the threshold, a heat wave is detected. Otherwise not.

def count_cells(ds):
    """Counts the number of cells in a data set"""
    x = ds.x.shape[0]
    y = ds.y.shape[0]
    return x * y


def count_true_cells_in_mask(ds_mask):
    """Counts the number of heat cells in a data set"""
    return ds_mask.sum().to_numpy()


def get_heat_waves_from_date(tas_daily_mean_masked,
                             tas_grtr_P95_ME,
                             start_date, 
                             nb_days=5, 
                             threshold=0.25,
                             verbose=False,
                             plotting=False,
                           ):
    """
    Based on a threshold, calculate the percentage of heat days over a 
    given number of days (e.g. 5 days) given a starting data.
    
    tas_daily_mean_masked: masked daily mean values
    tas_grtr_P95_ME: mask
    start_date: date at which to start looking for heatwaves
    nb_days: number of days necessary to count as a heatwave
    threshold: detection threshold for heatwave
    verbose: print results (boolean)
    plotting: plot results (boolean)
    """

    # year, month, day = start_date
    # start_date = datetime.date(year, month, day)

    count_cells_all_days = 0
    count_heat_cells_all_days = 0

    for day_i in range(nb_days):
        t = datetime.timedelta(days=-int((nb_days-1)/2)) + start_date + datetime.timedelta(days=day_i)
        t = t.strftime('%Y-%m-%d')

        tas_daily_mean_day = tas_daily_mean_masked.sel(time=t).squeeze()
        tas_grtr_P95_day = tas_grtr_P95_ME.sel(time=t).squeeze()

        count_cells_all_days += count_cells(tas_grtr_P95_day)
        count_heat_cells_all_days += count_true_cells_in_mask(tas_grtr_P95_day)

        if plotting is True:
            fig, ax_present = plt.subplots(figsize=[8, 4], 
                                           subplot_kw={"projection": ccrs.Robinson()})
            tas_daily_mean_day.plot(ax=ax_present, 
                                    transform=ccrs.PlateCarree(), 
                                    cmap="magma", robust=True)
            ax_present.coastlines()
            ax_present.add_feature(cart.feature.OCEAN, zorder=100, edgecolor='k')
            ax_present.set_title("Day " + str(day_i-int((nb_days-1)/2)) + " (" + t + ")")# - int((nb_days-1)/2)), t)
            plt.show()

    # Calculate percentage of heat cells over given number of days
    heat_cell_perc = count_heat_cells_all_days / count_cells_all_days
    heat_wave_detect = heat_cell_perc > threshold
    
    if verbose is True:
        print("Date: ", start_date.strftime('%Y-%m-%d'), " to ", t)
        print("Number of days: ", nb_days)
        print("Heat Wave Detected:", heat_wave_detect)
        print("Threshold of: ", str(threshold*100) + "%")
        print("Heat Cell Perc", str(np.round(heat_cell_perc, 2)*100) + "%")
        print("Nb Heat Cells", count_heat_cells_all_days)
        print("Nb Cells", count_cells_all_days)
        
    return heat_wave_detect, heat_cell_perc
        


### Main: Heatwave Detection

In [ ]:
w_day = 10
E = 200*200 # km^2, minimum area over which tas>P95 for heatwave to exist
X = 7 # days, minimum number of days for which tas(area)>P95 for heatwave to exist
year_range = list(np.arange(1979, 2015))
year_ofinterest = 2010
start_date = date(year_ofinterest, 7, 15)

tas_daily_mean_all, tas_grtr_P95_all = compute_tasAndP95mask(tas_historical, year_range)

tas_daily_mean = tas_daily_mean_all[year_range.index(year_ofinterest)]
tas_grtr_P95 = tas_grtr_P95_all[year_range.index(year_ofinterest)]


# We have chosen Middle East (ME) as our region of interest (where we want to detect heatwaves). 
# ME is approximately bounded within  [25°E to 65°E] and  [20°N to 40°N]

# Slice of lon and lat coordinates for middle east
lon_slice = slice(25, 65)  # Longitude Range: Approximately 25°E to 65°E
lat_slice = slice(20, 40)   # Latitude Range: Approximately 20°N to 40°N
tas_daily_mean_ME = tas_daily_mean.sel(x=lon_slice, y=lat_slice)
tas_grtr_P95_ME = tas_grtr_P95.sel(x=lon_slice, y=lat_slice)
tas_daily_mean_grtr_P95_ME = tas_daily_mean_ME.where(tas_grtr_P95_ME)

heatwave_over_region(tas_daily_mean_ME, 
                     tas_grtr_P95_ME, 
                     E, 
                     X, 
                     start_date)

#### Detect heatwaves in fixed region


In [ ]:
get_heat_waves_from_date(tas_daily_mean_grtr_P95_ME,
                                   tas_grtr_P95_ME,
                                   start_date, 
                                   nb_days=5, 
                                   threshold=0.25,
                                   verbose=True,
                                   plotting=True,
                                   )

In [ ]:
def annual_mean_heatwave(year_range,
                         E, X,
                         tas_daily_mean_all, 
                         tas_grtr_P95_all, 
                         lon_slice, 
                         lat_slice):
    n_hw=[]
    for year in year_range:
        tas_daily_mean = tas_daily_mean_all[year_range.index(year)]
        tas_grtr_P95 = tas_grtr_P95_all[year_range.index(year)]
        tas_daily_mean_regional = tas_daily_mean.sel(x=lon_slice, y=lat_slice)
        tas_grtr_P95_regional = tas_grtr_P95.sel(x=lon_slice, y=lat_slice)
        # tas_daily_mean_grtr_P95_regional = tas_daily_mean_regional.where(tas_grtr_P95_ME)

        sdate = date(year,7,1)   # start date
        edate = date(year,8,31)   # end date
        dates = [sdate+timedelta(days=x) for x in range(0,(edate-sdate).days, X)]
        n_total=0
        for start_date in dates:
            n = heatwave_over_region(tas_daily_mean_regional, tas_grtr_P95_regional, E, X, start_date, 
                                     plotting=True,mask_ocean=False, verbose = True, ret=True)
            n_total = n_total+n
        print(f"Total number of heatwaves in the year {year} are {n_total}")
        n_hw.append(n_total)
        # print(n_hw)
    return n_hw
    
    
    
# hw_over_years = annual_mean_heatwave(year_range,E, X,tas_daily_mean_all, tas_grtr_P95_all, lon_slice, lat_slice)
hw_over_years = annual_mean_heatwave([1979],E, X,tas_daily_mean_all, tas_grtr_P95_all, lon_slice, lat_slice)

In [ ]:
tas_daily_mean_all[0]

In [ ]:
years = range(1979, 2010)
hw = [25, 18, 19, 28, 20, 23, 20, 20, 15, 16, 18, 12, 17, 14, 26, 20, 23, 15, 19, 23, 18, 19, 17, 14, 18, 18, 22, 17, 14, 12, 13]

In [ ]:
fig, ax = plt.subplots(1, sharex=True)
fig.suptitle("Heatwaves over the Years")
ax.plot(years, hw)
ax.set_ylabel("Number of heatwaves")
ax.set_xlabel("Time")